### <center>Chestnut Bur Detection and Segmentation using MaskRCNN in PyTorch</center>

In [1]:
%matplotlib inline

import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, Subset, DataLoader
import torchvision
from torchvision import transforms as _transforms, tv_tensors
import torchvision.transforms.v2 as T
from torchvision.transforms import functional as F
import json
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from pathlib import Path
import gc
import os
from PIL import Image, ImageDraw, ImageOps

from segmentation_pytorch import engine

##### <center> Load the image and annotation data </center>

In [2]:
# load annotations from json file here: "C:\Users\exx\Downloads\Route 9 Orchard 3.v1-test_dataset.coco-segmentation\train\_annotations.coco.json"
annos = json.load(open("C:/Users/exx/Deep Learning/Chestnut_Bur_Instance_Segmentation/route9_orchard3/data/_annotations.coco.json"))

In [3]:
# convert the annos dict to a df
annos_df = pd.DataFrame(annos["annotations"])
df = pd.DataFrame()
df["tree_id"] = annos_df["image_id"].apply(lambda x: annos["images"][x]["file_name"].split("_")[0])
df["file_name"] = annos_df["image_id"].apply(lambda x: annos["images"][x]["file_name"])
df["file_name"] = df["file_name"].apply(lambda x: x.split("_")[0] + ".png")
categories = [cat["name"] for cat in annos["categories"]]
df["category_name"] = annos_df["category_id"].apply(lambda x: categories[x])
df["bbox"] = annos_df["bbox"].apply(lambda x: torch.tensor(x))
df["area"] = annos_df["area"].apply(lambda x: torch.tensor(x))
df["segmentation"] = annos_df["segmentation"].apply(lambda x: torch.tensor(x))
df["iscrowd"] = annos_df["iscrowd"]

# Filter df to remove treeIds not reviewed. 
reviewed_trees = [
    14, 44, 51, 60, 79, 91, 92, 117, 118, 146, 152, 171, 
    172, 210, 272, 276, 280, 286, 304, 309, 320, 329, 
    366, 369, 371, 392, 394, 405
]

df = df[df["tree_id"].isin([str(tree_id) for tree_id in reviewed_trees])]

In [4]:
image_dir = "C:/Users/exx/Deep Learning/Chestnut_Bur_Instance_Segmentation/route9_orchard3/data/images"
image_names = df["file_name"].unique()

In [5]:
# # plot each image with its canopy polygon mask and chestnut polygon masks

# os.makedirs("C:/Users/exx/Deep Learning/Chestnut_Bur_Instance_Segmentation/route9_orchard3/annotated_images", exist_ok=True)

# for i, tree_id in enumerate(df["tree_id"].unique()):
#     image_file = Path(image_dir) / df[df["tree_id"] == tree_id]["file_name"].values[0]
#     image = Image.open(image_file)
#     # get image width and height
#     width, height = image.size

#     # print(f"tree {tree_id} has a width of {width} and height of {height} pixels")

#     canopy_poly = df[(df["tree_id"] == tree_id) & (df["category_name"] == "Canopy")]["segmentation"].values
#     bur_poly = df[(df["tree_id"] == tree_id) & (df["category_name"] == "Chestnut-burr")]["segmentation"].values

#     canopy_poly = [np.array(poly[0]).reshape(-1, 2).astype(np.int32) for poly in canopy_poly]
#     bur_poly = [np.array(poly[0]).reshape(-1, 2).astype(np.int32) for poly in bur_poly]

#     # Create a drawing context
#     draw = ImageDraw.Draw(image)
    
#     for poly in canopy_poly:
#         draw.polygon([tuple(point) for point in poly], outline="purple", width=2)
#     for poly in bur_poly:
#         draw.polygon([tuple(point) for point in poly], outline="red")

#     # Save the image
#     image.save(f"C:/Users/exx/Deep Learning/Chestnut_Bur_Instance_Segmentation/route9_orchard3/annotated_images/{tree_id}_annotated.png", format='PNG')

##### <center> Pre-process and transform image and annotation data </center>

##### Adapted from: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#an-instance-segmentation-model-for-pennfudan-dataset

In [6]:
def mask_fill(mask, polys, color):
    from cv2 import fillPoly
    for poly in polys:
        fillPoly(mask, [poly], color)
    return mask

In [7]:
# Custom dataset loader (PyTorch) for loading images and annotation data
class ChestnutBurSegmentation(Dataset):
    """Custom Dataset for Chestnut Bur Segmentation in UAV Images"""

    def __init__(self, image_dir, df, transform = None):
        self.image_dir = image_dir
        self.df = df
        self.transform = transform
        self.unique_tree_ids = self.df["tree_id"].unique()

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tree_id = self.unique_tree_ids[idx]

        row = self.df[self.df["tree_id"] == tree_id].iloc[0]

        image_file = Path(self.image_dir) / row["file_name"]

        image = tv_tensors.Image(Image.open(image_file))

        height, width = image.shape[-2:]

        canopy_poly = self.df[(self.df["tree_id"] == tree_id) & (self.df["category_name"] == "Canopy")]["segmentation"].values
        bur_poly = self.df[(self.df["tree_id"] == tree_id) & (self.df["category_name"] == "Chestnut-burr")]["segmentation"].values


        canopy_poly = [np.array(poly[0]).reshape(-1, 2).astype(np.int32) for poly in canopy_poly]
        bur_poly = [np.array(poly[0]).reshape(-1, 2).astype(np.int32) for poly in bur_poly]


        # get canopy_bbox coords [xmin, ymin, xmax, ymax]
        xmin, ymin, xmax, ymax = min([poly[:, 0].min() for poly in canopy_poly]), min([poly[:, 1].min() for poly in canopy_poly]), max([poly[:, 0].max() for poly in canopy_poly]), max([poly[:, 1].max() for poly in canopy_poly])


        canopy_mask = np.zeros((height, width), dtype=np.uint8)
        canopy_mask = mask_fill(canopy_mask, canopy_poly, 1)

        bur_masks = []
        # One mask per bur. Store each bur mask in a list to stack later. 
        for poly in bur_poly:
            bur_mask = np.zeros((height, width), dtype=np.uint8)
            bur_mask = mask_fill(bur_mask, [poly], 2)
            bur_masks.append(bur_mask)

        background_mask = np.zeros((height, width, 1), dtype=np.uint8)
        bur_masks_stacked = np.stack(bur_masks, axis=-1)
        mask_image = np.concatenate([background_mask, bur_masks_stacked], axis=-1).transpose(2, 0, 1)
        labels = self.df[(self.df["tree_id"] == tree_id) & (self.df["category_name"] == "Chestnut-burr")]["category_name"].values
        labels = [categories.index(label) for label in labels]
        bboxes = self.df[(self.df["tree_id"] == tree_id) & (self.df["category_name"] == "Chestnut-burr")]["bbox"].values
        bboxes = [torch.tensor([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], dtype = torch.float32) for bbox in bboxes] # convert to xyxy format
        bboxes = torch.stack([bbox for bbox in bboxes], dim=0) # (n_objects, 4)
        area = self.df[(self.df["tree_id"] == tree_id) & (self.df["category_name"] == "Chestnut-burr")]["area"].values
        iscrowd = self.df[(self.df["tree_id"] == tree_id) & (self.df["category_name"] == "Chestnut-burr")]["iscrowd"].values

        # fill image background (outside of tree canopy)
        image = image * np.array(canopy_mask).astype(np.uint8)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(bboxes, format=tv_tensors.BoundingBoxFormat.XYXY, canvas_size=(height, width)) 
        target["labels"] = torch.as_tensor(labels, dtype = torch.int64) # (n_objects)
        target["image_id"] = int(tree_id)
        target["area"] = torch.stack([a.clone().detach().float() for a in area], dim=0) # (n_objects)
        target["iscrowd"] = torch.stack([torch.tensor(ic, dtype = torch.int64) for ic in iscrowd], dim=0) # (n_objects)

        image = T.Compose([T.ToDtype(torch.float32, scale=True),
                           T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])(image)
        
        image_and_mask = torch.cat((image, torch.tensor(mask_image, dtype=torch.float32)), dim=0)

        # crop image_and_mask to xmin, ymin, xmax, ymax
        image_and_mask = image_and_mask[:, (int(ymin)-50):(int(ymax)+50), (int(xmin)-50):(int(xmax)+50)] # pad by 50 in each direction
            
        if self.transform is not None:
            image_and_mask, target = self.transform(image_and_mask, target)
        
        image = image_and_mask[:image.shape[0], :, :] # slice out the transformed image
        mask_image = image_and_mask[image.shape[0]:, :].clone().detach().to(torch.uint8) # slice out the transformed mask

        target["masks"] = mask_image

        return image, target
        
    def collate_fn(batch):
        return tuple(zip(*batch))

    def __len__(self):
        return len(self.unique_tree_ids)

In [8]:
# define transforms for the dataset
def get_transform(train):
    transforms = []
    
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ClampBoundingBoxes()) # for segmentations too
    transforms.append(T.SanitizeBoundingBoxes()) ## for segmentations too

    return T.Compose(transforms)

In [9]:
sample_ds = ChestnutBurSegmentation(image_dir, df, get_transform(train = True))
sample_dl = DataLoader(sample_ds, batch_size = 4, shuffle = True, collate_fn = ChestnutBurSegmentation.collate_fn)

In [10]:
images, targets = next(iter(sample_dl))
images = [img for img in images]
targets = [{k: v for k, v in target.items()} for target in targets]

##### <center> Plot sample transformed images, targets, and masks </center>

In [ ]:
%matplotlib inline
tree_ids = [target["image_id"] for target in targets]

image = images[3].permute(1, 2, 0)
mask = targets[3]["masks"].permute(1, 2, 0)

plt.imshow(image)
plt.title("transformed image")
plt.show()

plt.imshow(mask.sum(axis=2), cmap="gray")
plt.title("chestnut bur masks")
plt.show()

# plot mask on image
plt.imshow(image)
plt.imshow(mask.sum(axis=2), cmap='gray', alpha=0.75)
plt.title("training sample")
plt.show()

In [12]:
# # save each transformed image with its canopy mask and chestnut polygon masks to a folder. Name image treeID.png

# os.makedirs("C:/Users/exx/Deep Learning/Chestnut_Bur_Instance_Segmentation/route9_orchard3/transformed_images", exist_ok=True)

# for i, (image, target) in enumerate(zip(images, targets)):

#     tree_id = target["image_id"]
#     image = image.permute(1, 2, 0)
#     mask = target["masks"].permute(1, 2, 0)

#     #plot mask on image and save to .png file
#     plt.imshow(image)
#     plt.imshow(mask.sum(axis=2), cmap='gray', alpha=0.75)
#     plt.savefig(f"C:/Users/exx/Deep Learning/Chestnut_Bur_Instance_Segmentation/route9_orchard3/transformed_images/{tree_id}_transformed.png")
#     plt.close()

##### <center> Construct MaskRCNN Model </center>

In [13]:
def get_instance_segmentation_model(num_classes):
    # Load a Mask R-CNN instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(
        weights=torchvision.models.detection.MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,
        weights_backbone=torchvision.models.ResNet50_Weights.DEFAULT
    )

    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Replace the pre-trained head with a new one to reflect the number of classes
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

    # Get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    # Replace the mask predictor with a new one
    model.roi_heads.mask_predictor = torchvision.models.detection.mask_rcnn.MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

In [ ]:
print(get_instance_segmentation_model(2))

##### <center> Tune model hyperparameters using bayesian optimization algo and hyperband scheduler </center>

##### adapted from: https://docs.ray.io/en/latest/tune/examples/bohb_example.html

In [15]:
# Bayesian Optimization HyperBand (BOHB) with HyperBand scheduler
# # https://proceedings.mlr.press/v80/falkner18a.html

import tempfile
import time
from pathlib import Path

import ray
from ray import train, tune
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB

import ConfigSpace as CS

In [16]:
# store indices in random order list for subsetting later
indices = torch.randperm(len(sample_ds)).tolist()

In [17]:
def load_data(image_dir = image_dir, df = df): 
    train_ds = ChestnutBurSegmentation(image_dir, df, get_transform(train = True))
    val_ds = ChestnutBurSegmentation(image_dir, df, get_transform(train = False))
    return train_ds, val_ds


def train_ChestnutBurSegmentation(search_space, indices):
    model = get_instance_segmentation_model(num_classes = 2) # background, chestnut bur
    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model) # train on multiple gpus if available
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, 
                                lr = search_space["lr"], 
                                momentum = search_space["momentum"], 
                                weight_decay = search_space["weight_decay"])

    warmup_factor = 1.0 / 1000
    warmup_iters = min(1000, int(len(indices)*0.8) - 1)

    # construct a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )
    
    # load existing checkpoint if available
    if train.get_checkpoint():
        loaded_checkpoint = train.get_checkpoint()
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            model.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)
    else:
        start_epoch = 0

    # load data
    train_ds, val_ds = load_data()
    
    train_ds = Subset(train_ds, indices[:-int(len(train_ds)*0.2)]) # first 80% of dataset for training
    val_ds = Subset(val_ds, indices[-int(len(train_ds)*0.2):-int(len(train_ds)*0.05)]) # next 15% of dataset for validation

    train_dl = DataLoader(train_ds, batch_size = search_space["batch_size"], shuffle = True, collate_fn = ChestnutBurSegmentation.collate_fn, num_workers=0, pin_memory=True)
    val_dl = DataLoader(val_ds, batch_size = 1, shuffle = False, collate_fn = ChestnutBurSegmentation.collate_fn, num_workers=0, pin_memory=True)

    # put large objects in Ray object store to help memory errors
    ray.put(model, train_ds, val_ds, val_dl)

    # Main training function
    for epoch in range(start_epoch, search_space["epochs"]):
        # train for one epoch, printing every 10 iterations
        train_logger, val_logger = engine.train_one_epoch(model, optimizer, train_dl, val_dl, device, epoch, print_freq=10, scaler=None)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        train_metrics, val_metrics = engine.evaluate(model, val_dl, device, train_dl)

        # save checkpoint
         # Here we save a checkpoint. It is automatically registered with Ray Tune
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            path = os.path.join(temp_checkpoint_dir, "checkpoint.pt")
            torch.save(
                (model.state_dict(), optimizer.state_dict()), path
            )
            checkpoint = train.Checkpoint.from_directory(temp_checkpoint_dir)
            train.report(
                {"train_loss": train_logger.loss.global_avg, # train loss,
                 "val_loss": val_logger.loss.global_avg, # val loss
                "train_mAP_50": train_metrics.coco_eval['segm'].stats[1], # train mAP@50
                "val_mAP_50": val_metrics.coco_eval['segm'].stats[1], # val mAP@50
                "train_mAR_100": train_metrics.coco_eval['segm'].stats[8], # train mAR@100
                "val_mAR_100": val_metrics.coco_eval['segm'].stats[8], # val mAR@100
                "epoch": epoch}, 
                checkpoint = checkpoint
            )
    
    print("Tuning Trial Complete!")


def test_best_model(best_result, indices):
    best_model = get_instance_segmentation_model(num_classes = 2) # background, chestnut bur

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_model.to(device)

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    model_state, _ = torch.load(checkpoint_path)
    best_model.load_state_dict(model_state)

    _, test_ds = load_data()
    test_ds = Subset(test_ds, indices[-int(len(test_ds)*0.05):]) # last 5% of dataset for testing
    test_dl = DataLoader(test_ds, batch_size = 1, shuffle = False, collate_fn = ChestnutBurSegmentation.collate_fn, num_workers=0, pin_memory=True)

    _, test_results = engine.evaluate(best_model, test_dl, device, train_data_loader=None)

    print(f'Best trial test set mAP_50: {test_results.coco_eval["segm"].stats[1]} and mAR_100: {test_results.coco_eval["segm"].stats[8]}')


def trial_dirname_creator(trial):
    return f"train_MAVdroneDataset_{trial.trial_id}"

In [ ]:
gc.collect()

torch.cuda.empty_cache()

def main(num_samples, indices):
    search_space = CS.ConfigurationSpace()
    search_space.add_hyperparameter(
        CS.Constant("epochs", 10)
    )
    search_space.add_hyperparameter(
        CS.UniformFloatHyperparameter("lr", lower=0.00009, upper=0.05)
    )
    search_space.add_hyperparameter(
        CS.UniformFloatHyperparameter("momentum", lower=0.00001, upper=0.99)
    )
    search_space.add_hyperparameter(
        CS.UniformFloatHyperparameter("weight_decay", lower=0.00001, upper=0.99)
    )
    search_space.add_hyperparameter(
        CS.CategoricalHyperparameter("batch_size", choices=[2, 4, 8])
    )

    # Extract the integer value of epochs
    max_epochs = search_space.get_hyperparameter("epochs").default_value
    
    # Ensure max_epochs is an integer
    max_epochs = int(max_epochs)

    algo = TuneBOHB(
        space=search_space,
        metric="mAP_50",
        mode="max",
    )
    algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = HyperBandForBOHB(
        time_attr="training_iteration",
        max_t=max_epochs,
        reduction_factor=4,
        stop_last_trials=False,
    )

    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_ChestnutBurSegmentation, indices=indices),
            resources = {"cpu": 48, "gpu": 1.0},
        ),
        run_config=train.RunConfig(
            name="bohb_exp",
            storage_path='C:/Users/exx/Documents/GitHub/Savanna-Institute/Drone-based_Chestnut_Detection/ray_results',
            stop={"training_iteration": max_epochs},
        ),
        tune_config = tune.TuneConfig(
            metric = "val_mAP_50",
            mode = "max",
            search_alg = algo,
            scheduler = scheduler,
            num_samples = int(num_samples),
            time_budget_s=600000,
            trial_dirname_creator=trial_dirname_creator
        ),
    )

    results = tuner.fit()

    best_result = results.get_best_result(metric = "val_mAP_50", mode = "max", scope = "all", filter_nan_and_inf=False)

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final training loss: {}".format(best_result.metrics["train_loss"]))
    print("Best trial final validation loss: {}".format(best_result.metrics["val_loss"]))
    print("Best trial final training mAP_50: {}".format(best_result.metrics["train_mAP_50"]))
    print("Best trial final validation mAP_50: {}".format(best_result.metrics["val_mAP_50"]))
    print("Best trial final training mAR_100: {}".format(best_result.metrics["train_mAR_100"]))
    print("Best trial final validation mAR_100: {}".format(best_result.metrics["val_mAR_100"]))

    test_best_model(best_result, indices)

    return best_result

if __name__ == "__main__": 
    best_trial = main(num_samples = 20, indices = indices)

In [ ]:
batch_size = best_trial.config["batch_size"]
lr = best_trial.config["lr"]
momentum = best_trial.config["momentum"]
weight_decay = best_trial.config["weight_decay"]

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2 # (background = 0, chestnut bur = 1)

# use our dataset and defined transformations
train_ds = ChestnutBurSegmentation(image_dir, df, get_transform(train = True))
val_ds = ChestnutBurSegmentation(image_dir, df, get_transform(train = False))

train_ds = Subset(train_ds, indices[:-int(len(train_ds)*0.2)]) # first 80% of dataset for training
val_ds = Subset(val_ds, indices[-int(len(train_ds)*0.2):-int(len(train_ds)*0.05)]) # next 15% of dataset for validation

train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True, collate_fn = ChestnutBurSegmentation.collate_fn)
valid_dl = DataLoader(val_ds, batch_size = 1, shuffle = False, collate_fn = ChestnutBurSegmentation.collate_fn)

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay
)

warmup_factor = 1.0 / 1000
warmup_iters = min(1000, int(len(indices)*0.8) - 1)

# construct a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, start_factor=warmup_factor, total_iters=warmup_iters
    )

num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_logger, val_logger = engine.train_one_epoch(model, optimizer, train_dl, valid_dl, device, epoch, print_freq=10, scaler=None)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    train_performance, val_performance = engine.evaluate(model, valid_dl, device, train_dl)

print("That's it!")

In [ ]:
# save model and weights to .pth
torch.save(model.state_dict(), f"C:/Users/exx/Documents/GitHub/Savanna-Institute/Drone-based_Chestnut_Detection/model_{time.strftime('%Y%m%d')}_{time.strftime('%H%M%S')}.pth")